# Fourier解析

黒木玄

2018-06-28

* Copyright 2018 Gen Kuroki
* License: MIT https://opensource.org/licenses/MIT

このファイルは次の場所できれいに閲覧できる:

* http://nbviewer.jupyter.org/github/genkuroki/Calculus/blob/master/12%20Fourier%20analysis.ipynb

このファイルは <a href="https://juliabox.com">Julia Box</a> で利用できる.

自分のパソコンに<a href="https://julialang.org/">Julia言語</a>をインストールしたい場合には

* <a href="http://nbviewer.jupyter.org/gist/genkuroki/81de23edcae631a995e19a2ecf946a4f">WindowsへのJulia言語のインストール</a>

を参照せよ.

論理的に完璧な説明をするつもりはない. 細部のいい加減な部分は自分で訂正・修正せよ.

$
\newcommand\eps{\varepsilon}
\newcommand\ds{\displaystyle}
\newcommand\Z{{\mathbb Z}}
\newcommand\R{{\mathbb R}}
\newcommand\C{{\mathbb C}}
\newcommand\QED{\text{□}}
\newcommand\root{\sqrt}
\newcommand\bra{\langle}
\newcommand\ket{\rangle}
\newcommand\d{\partial}
\newcommand\sech{\operatorname{sech}}
$

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Riemann-Lebesgueの定理" data-toc-modified-id="Riemann-Lebesgueの定理-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Riemann-Lebesgueの定理</a></span><ul class="toc-item"><li><span><a href="#階段函数" data-toc-modified-id="階段函数-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>階段函数</a></span></li><li><span><a href="#Riemann-Lebesgueの定理とその証明" data-toc-modified-id="Riemann-Lebesgueの定理とその証明-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Riemann-Lebesgueの定理とその証明</a></span></li></ul></li><li><span><a href="#Fourier変換の逆変換の収束" data-toc-modified-id="Fourier変換の逆変換の収束-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fourier変換の逆変換の収束</a></span><ul class="toc-item"><li><span><a href="#Fourier変換とFourier逆変換の定義" data-toc-modified-id="Fourier変換とFourier逆変換の定義-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Fourier変換とFourier逆変換の定義</a></span></li><li><span><a href="#Dirichlet核" data-toc-modified-id="Dirichlet核-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Dirichlet核</a></span></li><li><span><a href="#Riemannの局所性定理" data-toc-modified-id="Riemannの局所性定理-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Riemannの局所性定理</a></span></li><li><span><a href="#Fourier変換の逆変換の収束性-(Diniの条件)" data-toc-modified-id="Fourier変換の逆変換の収束性-(Diniの条件)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Fourier変換の逆変換の収束性 (Diniの条件)</a></span></li></ul></li><li><span><a href="#Fourier級数の収束" data-toc-modified-id="Fourier級数の収束-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fourier級数の収束</a></span><ul class="toc-item"><li><span><a href="#Fourier級数の定義" data-toc-modified-id="Fourier級数の定義-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Fourier級数の定義</a></span></li><li><span><a href="#Fourier展開のDirichlet核" data-toc-modified-id="Fourier展開のDirichlet核-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Fourier展開のDirichlet核</a></span></li><li><span><a href="#Fourier級数に関するRiemannの局所性定理" data-toc-modified-id="Fourier級数に関するRiemannの局所性定理-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Fourier級数に関するRiemannの局所性定理</a></span></li><li><span><a href="#Fourier級数の収束-(Diniの条件)" data-toc-modified-id="Fourier級数の収束-(Diniの条件)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Fourier級数の収束 (Diniの条件)</a></span></li></ul></li><li><span><a href="#Poissonの和公式" data-toc-modified-id="Poissonの和公式-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Poissonの和公式</a></span></li></ul></div>

In [1]:
using Plots
gr(); ENV["PLOTS_TEST"] = "true"
#clibrary(:colorcet)
clibrary(:misc)

function pngplot(P...; kwargs...)
    sleep(0.1)
    pngfile = tempname() * ".png"
    savefig(plot(P...; kwargs...), pngfile)
    showimg("image/png", pngfile)
end
pngplot(; kwargs...) = pngplot(plot!(; kwargs...))

showimg(mime, fn) = open(fn) do f
    base64 = base64encode(f)
    display("text/html", """<img src="data:$mime;base64,$base64">""")
end

using SymPy
#sympy[:init_printing](order="lex") # default
#sympy[:init_printing](order="rev-lex")

using SpecialFunctions
using QuadGK

## Riemann-Lebesgueの定理

### 階段函数

空でない区間 $I$ 上の函数で部分集合 $A$ 上で $1$, その外で $0$ になる函数を $\chi_A(x)$ と書く:

$$
\chi_A(x) = \begin{cases}
1 & (x\in A) \\
0 & (x\not\in A).
\end{cases}
$$

区間 $I$ に含まれる**長さが有限**の区間 $I_1,\ldots,I_n$ と数 $\alpha_1,\ldots,\alpha_n$ によって定まる函数

$$
s(x) = \sum_{i=1}^n \alpha_i\chi_A(x) \quad (x\in I)
$$

を区間 $I$ 上の**階段函数**と呼ぶことにする.

$f$ は区間 $I$ 上の函数であるとする. 任意の $\eps>0$ に対してある階段函数 $s(x)$ で

$$
\|f-s\|_1 = \int_I |f(x)-s(x)|\,dx < \eps
$$

を満たすものが存在するとき, $f$ は**階段函数で $L^1$ 近似可能**であるということにする. ここで $I=[a,b], [a,b), (a,b], (a,b)$ のとき $\int_I=\int_ab$ であるとする. $a=-\infty$ であっても $b=\infty$ であっても構わないとする.

**問題:** $\R=(-\infty,\infty)$ 上の連続函数 $f(x)$ で $\int_{-\infty}^\infty |f(x)|\,dx < \infty$ を満たすものは階段函数で $L^1$ 近似可能であることを示せ.

**解答例:** 任意に $\eps>0$ を取る $\int_{-\infty}^\infty |f(x)|\,dx < \infty$ より, 十分に $R>0$ を大きくすると, 

$$
\int_{-\infty}^{-R}|f(x)|\,dx + \int_R^\infty |f(x)|\,dx =
\int_{-\infty}^\infty |f(x)|\,dx - \int_{-R}^R |f(x)|\,dx < \frac{\eps}{2}.
$$

$f(x)$ は $[-R,R]$ 上で一様連続なので, 正の整数 $N$ を十分に大きくして, 区間 $[-R,R]$ を $N$ 等分したものを $I_1,\ldots,I_N$ として, $x_i\in I_i$ を任意に取ると,  

$$
|f(x)-f(x_i)|< \frac{\eps}{4R} \quad (x\in I_i,\ i=1,\ldots,N).
$$

ここで, 例えば $\Delta = 2R/N$, $a_i=R+i\Delta$, $I_1=[a_0,a_1)$, $\ldots$, $I_{N-1}=[a_{N-2},a_{N-1}])$, $I_N=[a_{N-1},a_N]$ とおいたと考えてよい. このとき階段函数 $s(x)$ を

$$
s(x)=\sum_{i=1}^N f(x_i)\chi_{I_i}(x)
$$

と定めると,

$$
|f(x)-f(x_i)| < \frac{\eps}{4R} \quad (-R\leqq x\leqq R).
$$

このとき

$$
\int_{-R}^R|f(x)-s(x)|\,dx \leqq \frac{\eps}{4R}2R = \frac{\eps}{2}.
$$

したがって,

$$
\begin{aligned}
\|f-s\|_1 &= \int_{-\infty}^\infty|f(x)-s(x)|\,dx 
\\ &=
\int_{-\infty}^{-R} |f(x)-s(x)|\,dx + \int_{-R}^R|f(x)-s(x)|\,dx + \int_R^\infty|f(x)-s(x)|\,dx 
\\ &=
\int_{-\infty}^{-R} |f(x)|\,dx + \int_{-R}^R|f(x)-s(x)|\,dx + \int_R^\infty|f(x)|\,dx 
\\ &< 
\frac{\eps}{2}+\frac{\eps}{2}=\eps.
\end{aligned}
$$

これで示すべきことが示された. $\QED$

**注意:** この注意はLebesgue積分論の知識を前提とする. 一般に区間上の可測函数 $f(x)$ で

$$
\int_I |f(x)|\,dx < \infty
$$

を満たすものを, $I$ 上の **$L^1$ 函数** または**可積分函数**(もしくは**積分可能函数**)と呼ぶ. $I$ 上 $L_1$ 函数は $I$ 上の階段函数で $L^1$ 近似可能であることが知られている. $\QED$

**補題:** 区間 $I$ 上の階段函数 $s(x)$ について

$$
\int_I s(x) e^{ipx}\,dx \to 0 \quad (|p|\to\infty).
$$

**証明:** $-\infty<a_j\leqq b_j<\infty$, $I_j=[a_j,b_j], [a_j,b_j), (a_j,b_j], (a_j,b_j)$ だとすると, $|p|\to\infty$ のとき,

$$
\int_I \chi_{I_j}(x)e^{ips}\,dx = 
\int_{a_j}^{b_j} e^{ipx}\,dx = 
\left[\frac{e^{ipx}}{ip}\right]_{x=a_j}^{x=b_j} =
\frac{e^{ipb_j} - e^{ipa_j}}{ip} \to 0.
$$

階段函数は $\chi_{I_j}(x)$ の有限一次結合なので補題の結果が成立することもわかる.  $\QED$

### Riemann-Lebesgueの定理とその証明

**定理(Riemann-Lebesgue):** $f$ は区間 $I$ 上の階段函数で $L^1$ 近似可能な函数であるとする. (Lebesgue積分論を知っている人は, $f(x)$ は区間 $I$ 上の $L^1$ 函数であると仮定してもよい.) このとき,

$$
\int_I f(x)e^{ipx}\,dx \to 0 \quad (|p|\to\infty).
$$

**証明:** 任意に $\eps>0$ を取る. $f(x)$ は階段函数で $L^1$ 近似可能なので, ある階段函数 $\ds s(x)=\sum_{i=1}^n a_i\chi_{I_i}(x)$ で

$$
\|f-s\|_1=\int_I |f(x)-s(x)|\,dx < \frac{\eps}{2}
$$

を満たすものが存在する. 上の補題より, ある $R>0$ が存在して, $|p|\geqq R$ ならば

$$
\left|\int_I s(x)e^{ipx}\,dx\right|<\frac{\eps}{2}.
$$

そのとき, 

$$
\begin{aligned}
\left|\int_I f(x)e^{ipx}\,dx\right| &=
\left|\int_I (f(x)-s(x))e^{ipx}\,dx  + \int_I s(x)e^{ipx}\,dx\right| 
\\ &\leqq
\left|\int_I (f(x)-s(x))e^{ipx}\,dx\right| + \left|\int_I s(x)e^{ipx}\,dx\right| 
\\ &\leqq
\int_I |f(x)-s(x)|\,dx + \left|\int_I s(x)e^{ipx}\,dx\right| 
\\ &=
\|f-s\|_1 + \left|\int_I s(x)e^{ipx}\,dx\right| <
\frac{\eps}{2}+\frac{\eps}{2} = \eps.
\end{aligned}
$$

これで $|p|\to\infty$ で $\int_I f(x)e^{ipx}\,dx\to 0$ となることがわかった. $\QED$

## Fourier変換の逆変換の収束

### Fourier変換とFourier逆変換の定義

$\R$ 上の函数 $f(x)$ の**Fourier変換** $\hat{f}(p)$ を次のように定義する:

$$
\hat{f}(p) = \int_\R f(x)e^{-2\pi ipx}\,dx.
$$

さらに $\R$ 上の函数 $g(p)$ の**Fourier逆変換** $\check{g}(x)$ を次のように定義する:

$$
\check{g}(x) = \int_\R g(p)e^{2\pi ipx}\,dp.
$$

$f(x)$ が $\R$ 上の $L^1$ 函数ならば, Lebesgueの収束定理より $\hat{f}(p)$ は $p$ の連続函数になり, Riemann-Lebegueの定理より $|p|\to\infty$ で $\hat{f}(p)\to 0$ となる. 

**注意:** Fourier変換とFourier逆変換を, 上の公式における $p$ を $p/(2\pi)$ で置き換えることによって, 

$$
\hat{f}(p) = \int_\R f(x)e^{-ipx}\,dx, \quad
\check{g}(p) = \frac{1}{2\pi}\int_\R g(p)e^{ipx}\,dp
$$

と定義するスタイルもよく使われている. $\QED$

この節では, $L^1$ 函数 $f(x)$ に対して, 

$$
f(x) = \lim_{N\to\infty} \int_{-N}^N \hat{f}(p)e^{ipx}\,dp 
$$

がいつ成立しているかを調べる. 

### Dirichlet核

**Dirichlet核** $D_N(x)$ を

$$
D_N(x) = \int_{-N}^N e^{2\pi ipx}\,dp = \left[\frac{e^{2\pi ipx}}{2\pi ix}\right]_{p=-N}^{p=N} =
\frac{e^{e\pi iNx}-e^{-2\pi iNx}}{2\pi ix} = \frac{\sin(2\pi Nx)}{\pi x}
$$

と定める. ただし, $D_N(0)=2N$ と定めておく. $D_N(x)$ は有界な偶函数になる.

このとき

$$
\begin{aligned}
\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp &=
\int_{-N}^N \left(\int_\R f(y)e^{-2\pi ipy}\,dy\right)e^{2\pi ipx}\,dp =
\int_\R f(y) \left(\int_{-N}^N e^{2\pi ip(x-y)}\,dp\right)\,dy
\\ &=
\int_\R f(y) D_N(x-y)\,dy =
\int_\R f(x+y) D_N(y)\,dy.
\end{aligned}
$$

最後の等号で $y$ を $x+y$ で置き換えて, $D_N(-y)=D_N(y)$ を使った.

**Dirichlet積分の公式:** 条件収束する広義積分に関する公式

$$
\ds\int_0^\infty\frac{\sin t}{t}\,dt=\frac{\pi}{2}
$$

より

$$
\int_{-\infty}^\infty D_N(x)\,dx = \frac{2}{\pi}\int_0^\infty \frac{\sin(2\pi Nx)}{x}\,dx =
\frac{2}{\pi}\int_0^\infty \frac{\sin(2\pi Nx)}{x}\,dx = \frac{2}{\pi}\frac{\pi}{2}=1.
$$

三番目の等号で $x=t/(2\pi N)$ と置換して計算した.  この公式は条件収束する積分の公式であることに注意せよ.

上の公式(Dirichlet積分の公式)については

* <a href="http://nbviewer.jupyter.org/github/genkuroki/Calculus/blob/master/09%20integration.ipynb">09 積分</a>

の<a href="http://nbviewer.jupyter.org/github/genkuroki/Calculus/blob/master/09%20integration.ipynb#Dirichlet積分とその一般化">Dirichlet積分とその一般化</a>の解説を参照せよ.

### Riemannの局所性定理

**Riemannの局所性定理:** $f$ は $\R$ 上の $L^1$ 函数であるとし, $\delta>0$ であるとする. このとき,

$$
\lim_{N\to\infty}\int_{-\infty}^{-\delta} f(x+y)D_N(y)\,dy = 0, \quad
\lim_{N\to\infty}\int_\delta^\infty f(x+y)D_N(y)\,dy = 0.
$$

これより, $\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp = \int_\R f(x+y) D_N(y)\,dy$ が $N\to\infty$ で収束することと, $\int_{-\delta}^\delta f(x+y) D_N(y)\,dy$ が $N\to\infty$ で収束することは同値であり, 収束する場合には同じ値に収束することがわかる.  後者が収束するか否かは $x$ の近くでの函数 $f$ の様子だけで決まることに注意せよ. 

**証明:**
$$
\begin{aligned}
&
\int_{-\infty}^{-\delta} f(x+y)D_N(y)\,dy = \int_{-\infty}^{-\delta} \frac{f(x+y)}{y} \sin(2\pi Ny)\,dy,
\\ &
\int_\delta^\infty f(x+y)D_N(y)\,dy = \int_{-\infty}^{-\delta} \frac{f(x+y)}{y} \sin(2\pi Ny)\,dy.
\end{aligned}
$$

であり, 区間 $(-\infty,-\delta)$ と $(\delta,\infty)$ のそれぞれの上で函数 $f(x+y)/y$ は $L^1$ 函数なので, Riemann-Lebesgueの定理より, これらは $N\to\infty$ で $0$ に収束する. $\QED$

### Fourier変換の逆変換の収束性 (Diniの条件)

**補題:** $\delta>0$ のとき,

$$
\lim_{N\to\infty}\int_{-\infty}^{-\delta} D_N(y)dy = 0, \quad
\lim_{N\to\infty}\int_\delta^\infty D_N(y)dy = 0. 
$$

**注意:** 条件収束する広義積分の極限に関する結果なのでRiemann-Lebesgueの定理を直接使用できない. $\QED$

**証明:** $D_N(y)$ は偶函数なので後者のみを示せば十分である. Dirichlet積分の公式より $\ds\lim_{N\to\infty}\int_0^{2\pi N\delta} \frac{\sin t}{t}\,dt=\frac{\pi}{2}$ なので, $\ds\lim_{N\to\infty}\int_{2\pi N\delta}^\infty \frac{\sin t}{\pi t}\,dt=0$ となることがわかる. ゆえに, $x=t/(2\pi N)$ とおくと, 

$$
\int_\delta^\infty D_N(y)dy = \int_\delta^\infty \frac{\sin(2\pi Nx)}{\pi x}\,dx =
\int_{2\pi N\delta}^\infty \frac{\sin t}{\pi t}\,dt\to 0 \quad (N\to\infty).
\qquad\QED
$$

**定理(Diniの条件):** $f$ は $\R$ 上の $L^1$ 函数であり, $x\in \R$ であるとする.とする. さらに, ある $\delta>0$ が存在して, 

$$
\int_0^\delta \frac{|(f(x+y) + f(x-y) - 2f(x)|}{y}\,dy <\infty
\tag{$*$}
$$

が成立していると仮定する. この条件($*$)を**Diniの条件**と呼ぶ. このとき, 

$$
\lim_{N\to\infty}\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp = \lim_{N\to\infty}\int_\R f(x+y) D_N(y)\,dy = f(x).
$$

**証明:** $\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp - f(x)$ が $N\to\infty$ で $0$ に収束することを示せばよい. 

$$
\begin{aligned}
\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp - f(x) &=
\int_\R f(x+y) D_N(y)\,dy - f(x) \int_{-\infty}^\infty D_N(y)\,dy
\\ &=
\int_{-\infty}^\infty (f(x+y)-f(x)) D_N(y)\,dy
\\ &=
\int_0^\infty (f(x+y)-f(x)) D_N(y)\,dy +
\int_{-\infty}^0 (f(x+y)-f(x)) D_N(y)\,dy
\\ &=
\int_0^\infty (f(x+y)-f(x)) D_N(y)\,dy +
\int_0^\infty (f(x-y)-f(x)) D_N(y)\,dy
\\ &=
\int_0^\infty (f(x+y)+f(x-y)-2f(x)) D_N(y)\,dy
\\ &=
\int_0^\infty \frac{f(x+y)+f(x-y)-2f(x)}{y} \frac{\sin(2\pi N y)}{\pi}\,dy = I_N + J_N.
\end{aligned}
$$

ここで, 

$$
\delta > 0, \quad
I_N = \int_0^\delta \frac{f(x+y)+f(x-y)-2f(x)}{y} \frac{\sin(2\pi N y)}{\pi}\,dy, \quad
J_N = \int_\delta^\infty \frac{f(x+y)+f(x-y)-2f(x)}{y} \frac{\sin(2\pi N y)}{\pi}\,dy.
$$

$N\to\infty$ のとき, Riemannの局所性定理と上の補題より $J_N\to 0$ となり, Diniの条件の仮定とRiemann-Lebesgueの定理より $I_N\to 0$ となる. これで示したいことが示せた. $\QED$

**例:** $f$ は $\R$ 上の $L^1$ 函数であるとする. $y\searrow 0$ のとき $f(x+y), f(x-y)$ は収束していると仮定し, それぞれの収束先を $f(x+0), f(x-0)$ と書くことにする. さらに $y\searrow 0$ で $\ds\frac{f(x+y)-f(x+0)}{y}$ と $\ds\frac{f(x-y)-f(x-0)}{y}$ は収束していると仮定する. 

このとき, 十分小さな $\delta>0$ を取ると, $\ds\frac{f(x+y)+f(x-y)-(f(x+0)+f(x-0))}{y}$ は $0<y<\delta$ で有界になり, 特にそこで可積分になる. ゆえに $f(x)=(f(x+0)+f(x-0))/2$ ならば, Diniの条件が満たされており, 上の定理より,

$$
\lim_{N\to\infty}\int_{-N}^N \hat{f}(p)e^{2\pi ipx}\,dp = \lim_{N\to\infty}\int_\R f(x+y) D_N(y)\,dy = f(x).
$$

特に $f$ が $x$ で微分可能ならばこれが成立している. さらにもしもFourier変換 $\hat{f}(p)$ が可積分でかつDiniの条件が満たされているならば,

$$
\hat{f}(p)=\int_{-\infty}^\infty f(x)e^{-2\pi ipx}\,dx
\implies
\int_{-\infty}^\infty \hat{f}(p)e^{2\pi ipx}\,dp = f(x)
$$

が絶対収束する積分の公式として成立している. $\QED$

## Fourier級数の収束

Fourier変換の逆変換の収束の議論とほぼ同じ議論を以下では行う.

### Fourier級数の定義

$f(x)$ は周期 $1$ を持つ $\R$ 上の函数であると仮定する: $f(x+1)=f(x)$.  このとき, 整数 $k\in\Z$ に対して, $f(x)$ のFourier係数 $a_k(f)$ を

$$
a_k(f) = \int_0^1 f(x)e^{-2\pi i kx}\,dx
$$

と定める. そして, 

$$
\sum_{k\in\Z} a_k(f) e^{2\pi ikx}
$$

を $f(x)$ の**Fourier級数展開**と呼ぶ. 

### Fourier展開のDirichlet核

**Dirichlet核** $D_N(x)$ を

$$
D_N(x) = \sum_{k=-N}^N e^{2\pi ikx} = e^{-2\pi iNx}\frac{e^{2\pi i(2N+1)x}-1}{e^{2\pi i x}-1} =
\frac{\sin(\pi(2N+1)x)}{\sin(\pi x)}
$$

と定める. ただし, $x\in\Z$ のときには $D_N(x)=2N+1$ と定めておく.  $D_N(x)$ は有界な偶函数で周期 $1$ を持つ.

このとき, 

$$
\begin{aligned}
\sum_{k=-N}^N a_k(f)e^{2\pi ikx} &=
\sum_{k=-N}^N \left(\int_0^1 f(y)e^{2\pi i ky}\,dy\right)e^{2\pi ikx} =
\int_0^1 f(y)\left(\sum_{k=-N}^N e^{2\pi i k(x-y)}\,dx\right)\,dy 
\\ &=
\int_0^1 f(y)D_N(x-y)\,dy =
\int_{-1/2}^{1/2} f(x+y)D_N(y)\,dy.
\end{aligned}
$$

最後の等号で $y$ を $x+y$ で置換し, $D_N(-y)=D_N(y)$ と $f(x+y)D_N(y)$ が $y$ について周期 $1$ を持つ函数になることを用いた.

$$
\int_0^1 D_N(y)\,dy = \sum_{k=-N}^N\int_0^1 e^{2\pi iky} \,dy = \int_0^1 1 \,dy = 1.
$$

### Fourier級数に関するRiemannの局所性定理

**Riemannの局所性定理:** $f$ は $\R$ 上の周期 $1$ を持つ函数であり, $[0,1)$ 上で可積分であると仮定し, $0<\delta<1/2$ であるとする. このとき, 

$$
\lim_{N\to\infty}\int_{-1/2}^{-\delta} f(x+y)D_N(y)\,dy = 0, \quad
\lim_{N\to\infty}\int_\delta^{1/2} f(x+y)D_N(y)\,dy = 0.
$$

これより, $\sum_{k=-N}^N a_n(f)e^{2\pi ikx} = \int_{-1/2}^{1/2} f(x+y) D_N(y)\,dy$ が $N\to\infty$ で収束することと, $\int_{-\delta}^\delta f(x+y) D_N(y)\,dy$ が $N\to\infty$ で収束することは同値であり, 収束する場合には同じ値に収束することがわかる.  後者が収束するか否かは $x$ の近くでの函数 $f$ の様子だけで決まることに注意せよ. 

**証明:** $0<\delta<1/2$ のとき, 

$$
\begin{aligned}
&
\int_{-1/2}^{-\delta} f(x+y)D_N(y)\,dy = \int_{-1/2}^{-\delta} \frac{f(x+y)}{\sin(\pi y)}\sin(\pi(2N+1)y\,dy,
\\ &
\int_\delta^{1/2} f(x+y)D_N(y)\,dy = \int_\delta^{1/2} \frac{f(x+y)}{\sin(\pi y)}\sin(\pi(2N+1)y\,dy.
\end{aligned}
$$

区間 $(-1/2,-\delta)$, $(\delta,1/2)$ で $\ds \frac{f(x+y)}{\sin(\pi y)}$ は可積分なので, Riemann-Lebesgueの定理より, これらは $N\to\infty$ で $0$ に収束する. $\QED$

### Fourier級数の収束 (Diniの条件)

**定理(Diniの条件):** $f$ は $\R$ 上の周期 $1$ を持つ函数であり, 区間 $[0,1)$ 上で可積分であると仮定し, $x\in \R$ であるとする. さらに, $0<\delta<1/2$ を満たすある実数 $\delta$ が存在して,  

$$
\int_0^\delta \frac{|(f(x+y) + f(x-y) - 2f(x)|}{y}\,dy <\infty
\tag{$*$}
$$

が成立していると仮定する. この条件($*$)を**Diniの条件**と呼ぶ. このとき, 

$$
\lim_{N\to\infty}\sum_{k=-N}^N a_n(f)e^{2\pi ikx} = \lim_{N\to\infty}\int_{-1/2}^{1/2} f(x+y) D_N(y)\,dy = f(x).
$$

**証明:** Fourier変換の場合と同様に計算すると, 

$$
\begin{aligned}
\sum_{k=-N}^N a_n(f)e^{2\pi ikx} - f(x) &=
\int_{-1/2}^{1/2} f(x+y) D_N(y)\,dy - \int_{-1/2}^{1/2}D_N(y)\,dy
\\ &=
\int_{-1/2}^{1/2} (f(x+y)-f(x)) D_N(y)\, dy
\\ &=
\int_0^{1/2} (f(x+y)-f(x)) D_N(y)\, dy +
\int_{-1/2}^0 (f(x+y)-f(x)) D_N(y)\, dy
\\ &=
\int_0^{1/2} (f(x+y)-f(x)) D_N(y)\, dy +
\int_0^{1/2} (f(x-y)-f(x)) D_N(y)\, dy
\\ &=
\int_0^{1/2} \frac{f(x+y)+f(x-y)-2f(x)}{y} \frac{y}{\sin(\pi y)} \sin(\pi(2N+1)y)\,dy.
\end{aligned}
$$

Diniの条件より, Riemann-Lebesgueの定理を使用できるので, これは $N\to\infty$ で $0$ に収束することがわかる. $\QED$

## Poissonの和公式

Fourier級数の収束とFourier変換の定義から, **Poissonの和公式**が得られることを説明しよう.

**定義(急減少函数):** $\R$ 上の $C^\infty$ 函数 $f$ で, 任意の非負の整数 $m,n$ に対して, $|x|\to\infty$ のとき $f^{(m)}(x)x^n\to 0$ となるものを**急減少函数**と呼ぶ. $\QED$

**定理(Poissonの和公式)** $f$ は $\R$ 上の急減少函数であると仮定する. このとき, 次の公式が成立している:

$$
\sum_{m\in\Z} f(m) = \sum_{n\in\Z} \hat{f}(n)
$$

**略証:** $\ds g(x) = \sum_{m\in\Z} f(x+m)$ とおくと, $g(x)$ は周期 $1$ を持つ $C^\infty$ 函数になる. ゆえに,

$$
g(x) = \sum_{n\in\Z} a_n(g)e^{2\pi inx} \quad (x\in \R).
$$

そして, 

$$
\begin{aligned}
a_n(g) &= \int_0^1 \sum_{m\in\Z}f(x+m)e^{-2\pi inx}\,dx =
\sum_{m\in\Z} \int_0^1 f(x+m)e^{-2\pi inx}\,dx 
\\ &=
\sum_{m\in\Z} \int_m^{m+1} f(x)e^{-2\pi inx}\,dx =
\int_{-\infty}^\infty f(x)e^{-2\pi inx}\,dx = \hat{f}(n).
\end{aligned}
$$

以上をまとめると,

$$
g(x) = \sum_{m\in\Z} f(x+m) = \sum_{n\in\Z} \hat{f}(n)e^{2\pi inx} \quad (x\in\R).
$$

この等式で $x=0$ をおけば欲しい公式が得られる. $\QED$

**問題:** $t>0$ に対して, 次の公式を示せ:

$$
\sum_{m\in\Z} \frac{e^{-\pi(x-m)^2/t}}{\sqrt{t}} = \sum_{n\in\Z} e^{-\pi n^2 t + 2\pi inx}.
$$

**注意:** この問題の左辺と右辺は楕円テータ函数と呼ばれる特殊函数の特別な場合になっている. 左辺は熱方程式の基本解を周期函数になるように足し上げたものになっている. そのことから, この公式の両辺は周期境界条件のもとでの熱方程式の基本解になっていることがわかる. 楕円テータ函数については

* David Mumford, <a href="http://www.dam.brown.edu/people/mumford/alg_geom/papers/Tata1.pdf">Tata Lectures on Theta I</a>, Reprint of the 1983 Edition

が非常に面白くてかつ読み易い教科書であり, 論文などでもよく引用されている. $\QED$

**解答例:** $\ds f(x)=\frac{e^{-\pi x^2/t}}{\sqrt{t}}$ とおくと, $f(x)$ が偶函数であることと, Poissonの和公式の証明より,

$$
\sum_{m\in\Z} \frac{e^{-\pi(x-m)^2/t}}{\sqrt{t}} =
\sum_{m\in\Z}f(x-m)=\sum_{m\in\Z}f(x+m)=\sum_{n\in\Z}\hat{f}(n)e^{2\pi inx}.
$$

そして, 公式

$$
\int_{-\infty}^\infty e^{-y^2/a}e^{-ipy}\,dy = \sqrt{\pi a}e^{-ap^2/4}
$$

より, $x=y/(2\pi)$ とおくと, 

$$
\hat{f}(n) = \int_{-\infty}^\infty \frac{e^{-\pi x^2/t}}{\sqrt{t}} e^{-2\pi inx}\,dx =
\int_{-\infty}^\infty \frac{e^{-y^2/(4\pi t)}}{\sqrt{t}} e^{-iny}\,\frac{dy}{2\pi} =
\frac{\sqrt{4\pi t\,\pi}}{2\pi\sqrt{t}}e^{-4\pi t n^2/4} =
e^{-\pi n^2 t}.
$$

以上をまとめると示したい公式が得られる. $\QED$

**問題:** $t>0$ に対して, $\ds \Theta(t)=\sum_{n\in\Z} e^{-\pi n^2 t} = 1 + 2\sum_{n=1}^\infty e^{-\pi n^2 t}$ とおくと, 

$$
\Theta\left(\frac{1}{t}\right) = \sqrt{t}\;\Theta(t)
$$

が成立することを示せ. 

**注意:** この公式はRiemannのゼータ函数の函数等式を証明するときに使われる. この問題の結果は楕円テータ函数のモジュラー変換性の特別な場合になっている. モジュラー変換性を持つ函数は数論的にも極めて重要な数学的対象である. $\QED$

**解答例:**  $\ds f(x)=\frac{e^{-\pi x^2/t}}{\sqrt{t}}$ にPoissonの和公式を適用すればこの公式が得られる. すなわち, 上の問題の結果で $x=0$ とおけば示したい公式が得られる. $\QED$